# Simulated Annealing Optimizer

Implementing SA with:
1. Quick wins: N=1 at 45° rotation
2. 8 move types from jonathanchan kernel
3. Adaptive neighborhood (crystallization factor)
4. Prioritization by N (small N gets more iterations)
5. ZERO TOLERANCE overlap checking

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import math
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Imports complete")

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

In [ ]:
def has_overlap(trees):
    """Check if any trees overlap using ZERO tolerance."""
    if len(trees) <= 1:
        return False
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def check_single_overlap(trees, idx):
    """Check if tree at idx overlaps with any other tree."""
    poly = trees[idx].polygon
    for j, other in enumerate(trees):
        if j == idx:
            continue
        if poly.intersects(other.polygon) and not poly.touches(other.polygon):
            return True
    return False

def get_bounding_box(trees):
    """Get bounding box dimensions."""
    all_coords = []
    for t in trees:
        coords = np.asarray(t.polygon.exterior.xy).T / 1e15
        all_coords.append(coords)
    all_coords = np.concatenate(all_coords)
    
    min_x, min_y = all_coords.min(axis=0)
    max_x, max_y = all_coords.max(axis=0)
    
    return min_x, min_y, max_x, max_y

def get_side_length(trees):
    """Calculate the side length of the square bounding box."""
    min_x, min_y, max_x, max_y = get_bounding_box(trees)
    return max(max_x - min_x, max_y - min_y)

def get_score(trees, n=None):
    """Calculate score for configuration."""
    side = get_side_length(trees)
    score = side ** 2
    if n is not None:
        score /= n
    return score

print("Helper functions defined")

In [ ]:
# Quick win: Find optimal rotation for N=1
print("Finding optimal rotation for N=1...")
best_score_n1 = float('inf')
best_angle_n1 = 0

for angle in range(0, 91, 1):  # Test 0 to 90 degrees
    tree = ChristmasTree(0, 0, angle)
    score = get_score([tree], 1)
    if score < best_score_n1:
        best_score_n1 = score
        best_angle_n1 = angle

print(f"N=1: Best angle = {best_angle_n1}°, Score = {best_score_n1:.6f}")
print(f"N=1 at 0°: Score = {get_score([ChristmasTree(0, 0, 0)], 1):.6f}")
print(f"Improvement: {get_score([ChristmasTree(0, 0, 0)], 1) - best_score_n1:.6f}")

In [ ]:
def find_best_grid_trees(n: int) -> tuple:
    """Find best grid arrangement for n trees (from baseline)."""
    best_score, best_trees = float("inf"), None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            all_trees = []
            rest = n
            r = 0
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
    
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else Decimal("0.7") / 2
                y = r // 2 * Decimal("1.0") if r % 2 == 0 else (Decimal("0.8") + (r - 1) // 2 * Decimal("1.0"))
                row_trees = [ChristmasTree(center_x=Decimal("0.7") * i + x_offset, center_y=y, angle=angle) for i in range(m)]
                all_trees.extend(row_trees)
    
                r += 1
            
            if len(all_trees) != n:
                continue
                
            xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in all_trees])
            min_x, min_y = xys.min(axis=0)
            max_x, max_y = xys.max(axis=0)
            score = max(max_x - min_x, max_y - min_y) ** 2
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print("Grid placement function defined")

In [ ]:
def simulated_annealing(trees, n, max_iter=1000, T0=1.0, alpha=0.95, move_scale=0.1, rot_scale=30.0):
    """
    Simulated Annealing optimizer with 8 move types.
    
    Move types:
    0: Random translation
    1: Centroid move (toward center)
    2: Random rotation
    3: Combined translation + rotation
    4: Swap positions of two trees
    5: Bbox center move
    6: Corner tree focus
    7: Coordinated move (two adjacent trees)
    """
    if len(trees) <= 1:
        return trees, get_score(trees, n)
    
    # Copy trees to avoid modifying original
    current_trees = [ChristmasTree(str(t.center_x), str(t.center_y), str(t.angle)) for t in trees]
    current_score = get_score(current_trees, n)
    
    best_trees = [ChristmasTree(str(t.center_x), str(t.center_y), str(t.angle)) for t in current_trees]
    best_score = current_score
    
    T = T0
    no_improve = 0
    
    # Adaptive step sizes
    pos_step = move_scale
    rot_step = rot_scale
    
    for iteration in range(max_iter):
        # Temperature scaling
        scale = T / T0
        
        # Choose move type
        move_type = random.randint(0, 7)
        
        # Select tree(s) to modify
        i = random.randint(0, len(current_trees) - 1)
        
        # Store original values
        orig_x = current_trees[i].center_x
        orig_y = current_trees[i].center_y
        orig_angle = current_trees[i].angle
        
        # Calculate centroid and bbox for some moves
        min_x, min_y, max_x, max_y = get_bounding_box(current_trees)
        cx = sum(float(t.center_x) for t in current_trees) / len(current_trees)
        cy = sum(float(t.center_y) for t in current_trees) / len(current_trees)
        bbox_cx = (min_x + max_x) / 2
        bbox_cy = (min_y + max_y) / 2
        
        # Apply move
        if move_type == 0:  # Random translation
            dx = (random.random() - 0.5) * 2 * pos_step * scale
            dy = (random.random() - 0.5) * 2 * pos_step * scale
            new_x = float(orig_x) + dx
            new_y = float(orig_y) + dy
            current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(orig_angle))
            
        elif move_type == 1:  # Centroid move
            dx = (cx - float(orig_x)) * random.random() * scale * 0.3
            dy = (cy - float(orig_y)) * random.random() * scale * 0.3
            new_x = float(orig_x) + dx
            new_y = float(orig_y) + dy
            current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(orig_angle))
            
        elif move_type == 2:  # Random rotation
            da = (random.random() - 0.5) * 2 * rot_step * scale
            new_angle = float(orig_angle) + da
            current_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(new_angle))
            
        elif move_type == 3:  # Combined translation + rotation
            dx = (random.random() - 0.5) * 2 * pos_step * scale
            dy = (random.random() - 0.5) * 2 * pos_step * scale
            da = (random.random() - 0.5) * 2 * rot_step * scale
            new_x = float(orig_x) + dx
            new_y = float(orig_y) + dy
            new_angle = float(orig_angle) + da
            current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(new_angle))
            
        elif move_type == 4:  # Swap positions
            if len(current_trees) >= 2:
                j = random.randint(0, len(current_trees) - 1)
                while j == i:
                    j = random.randint(0, len(current_trees) - 1)
                orig_j_x = current_trees[j].center_x
                orig_j_y = current_trees[j].center_y
                orig_j_angle = current_trees[j].angle
                current_trees[i] = ChristmasTree(str(orig_j_x), str(orig_j_y), str(orig_angle))
                current_trees[j] = ChristmasTree(str(orig_x), str(orig_y), str(orig_j_angle))
            else:
                continue
                
        elif move_type == 5:  # Bbox center move
            dx = (bbox_cx - float(orig_x)) * random.random() * scale * 0.3
            dy = (bbox_cy - float(orig_y)) * random.random() * scale * 0.3
            new_x = float(orig_x) + dx
            new_y = float(orig_y) + dy
            current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(orig_angle))
            
        elif move_type == 6:  # Corner tree focus - move corner trees inward
            # Find if this tree is at a corner
            tree_min_x = float(current_trees[i].polygon.bounds[0]) / 1e15
            tree_max_x = float(current_trees[i].polygon.bounds[2]) / 1e15
            tree_min_y = float(current_trees[i].polygon.bounds[1]) / 1e15
            tree_max_y = float(current_trees[i].polygon.bounds[3]) / 1e15
            
            is_corner = (abs(tree_min_x - min_x) < 0.01 or abs(tree_max_x - max_x) < 0.01 or
                        abs(tree_min_y - min_y) < 0.01 or abs(tree_max_y - max_y) < 0.01)
            
            if is_corner:
                # Move toward center
                dx = (bbox_cx - float(orig_x)) * random.random() * scale * 0.2
                dy = (bbox_cy - float(orig_y)) * random.random() * scale * 0.2
            else:
                dx = (random.random() - 0.5) * 2 * pos_step * scale
                dy = (random.random() - 0.5) * 2 * pos_step * scale
            new_x = float(orig_x) + dx
            new_y = float(orig_y) + dy
            current_trees[i] = ChristmasTree(str(new_x), str(new_y), str(orig_angle))
            
        elif move_type == 7:  # Coordinated move
            if len(current_trees) >= 2:
                j = (i + 1) % len(current_trees)
                dx = (random.random() - 0.5) * 2 * pos_step * scale
                dy = (random.random() - 0.5) * 2 * pos_step * scale
                orig_j_x = current_trees[j].center_x
                orig_j_y = current_trees[j].center_y
                orig_j_angle = current_trees[j].angle
                current_trees[i] = ChristmasTree(str(float(orig_x) + dx), str(float(orig_y) + dy), str(orig_angle))
                current_trees[j] = ChristmasTree(str(float(orig_j_x) + dx), str(float(orig_j_y) + dy), str(orig_j_angle))
            else:
                continue
        
        # Check for overlap
        if has_overlap(current_trees):
            # Revert move
            if move_type == 4:  # Swap
                current_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(orig_angle))
                current_trees[j] = ChristmasTree(str(orig_j_x), str(orig_j_y), str(orig_j_angle))
            elif move_type == 7:  # Coordinated
                current_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(orig_angle))
                current_trees[j] = ChristmasTree(str(orig_j_x), str(orig_j_y), str(orig_j_angle))
            else:
                current_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(orig_angle))
            # Decrease step size (crystallization)
            pos_step *= 0.99
            rot_step *= 0.99
            continue
        
        # Calculate new score
        new_score = get_score(current_trees, n)
        
        # Accept or reject
        delta = new_score - current_score
        if delta < 0 or random.random() < math.exp(-delta / T):
            current_score = new_score
            # Increase step size (exploration)
            pos_step = min(pos_step * 1.01, move_scale)
            rot_step = min(rot_step * 1.01, rot_scale)
            
            if current_score < best_score:
                best_score = current_score
                best_trees = [ChristmasTree(str(t.center_x), str(t.center_y), str(t.angle)) for t in current_trees]
                no_improve = 0
            else:
                no_improve += 1
        else:
            # Revert move
            if move_type == 4:  # Swap
                current_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(orig_angle))
                current_trees[j] = ChristmasTree(str(orig_j_x), str(orig_j_y), str(orig_j_angle))
            elif move_type == 7:  # Coordinated
                current_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(orig_angle))
                current_trees[j] = ChristmasTree(str(orig_j_x), str(orig_j_y), str(orig_j_angle))
            else:
                current_trees[i] = ChristmasTree(str(orig_x), str(orig_y), str(orig_angle))
            # Decrease step size
            pos_step *= 0.99
            rot_step *= 0.99
            no_improve += 1
        
        # Cool down
        T *= alpha
        
        # Reheat if stagnant
        if no_improve > 100:
            T = min(T * 3.0, T0 * 0.7)
            no_improve = 0
    
    return best_trees, best_score

print("SA function defined")

In [ ]:
# Test SA on small N
print("Testing SA on N=5...")
_, grid_trees = find_best_grid_trees(5)
grid_score = get_score(grid_trees, 5)
print(f"Grid score for N=5: {grid_score:.6f}")

sa_trees, sa_score = simulated_annealing(grid_trees, 5, max_iter=500)
print(f"SA score for N=5: {sa_score:.6f}")
print(f"Improvement: {grid_score - sa_score:.6f}")
print(f"Has overlap: {has_overlap(sa_trees)}")

In [ ]:
# Generate all solutions with SA optimization
print("Generating solutions for N=1 to 200 with SA optimization...")
print("Priority: N=1-20 (3x iter), N=21-50 (2x iter), N=51-100 (1.5x iter), N=101-200 (1x iter)")

solutions = []
base_iter = 300  # Base iterations

for n in tqdm(range(1, 201)):
    if n == 1:
        # Use optimal rotation for N=1
        trees = [ChristmasTree(0, 0, best_angle_n1)]
        score = get_score(trees, 1)
    else:
        # Get grid baseline
        _, grid_trees = find_best_grid_trees(n)
        grid_score = get_score(grid_trees, n)
        
        # Determine iterations based on N
        if n <= 20:
            iterations = base_iter * 3
        elif n <= 50:
            iterations = base_iter * 2
        elif n <= 100:
            iterations = int(base_iter * 1.5)
        else:
            iterations = base_iter
        
        # Run SA
        sa_trees, sa_score = simulated_annealing(grid_trees, n, max_iter=iterations)
        
        # Use better solution
        if sa_score < grid_score and not has_overlap(sa_trees):
            trees = sa_trees
            score = sa_score
        else:
            trees = grid_trees
            score = grid_score
    
    solutions.append((score * n, trees))  # Store raw score (not divided by n)

print(f"Generated {len(solutions)} solutions")

In [ ]:
# Calculate overall score
overall_score = sum(score / n for n, (score, _) in enumerate(solutions, 1))
print(f"Overall score: {overall_score:.6f}")

# Compare with baseline
baseline_score = 88.329998
print(f"Baseline score: {baseline_score:.6f}")
print(f"Improvement: {baseline_score - overall_score:.6f}")

# Show breakdown by N ranges
print("\nScore breakdown by N range:")
for start, end in [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]:
    range_score = sum(solutions[n-1][0] / n for n in range(start, end+1))
    print(f"  N={start}-{end}: {range_score:.6f}")

In [ ]:
# Validate no overlaps
print("Validating no overlaps...")
has_any_overlap = False
for n, (score, trees) in enumerate(solutions, 1):
    if has_overlap(trees):
        print(f"OVERLAP DETECTED at N={n}!")
        has_any_overlap = True

if not has_any_overlap:
    print("No overlaps detected in any configuration!")
else:
    print("WARNING: Overlaps detected - submission will fail!")

In [ ]:
# Create submission
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n, (_, all_trees) in enumerate(solutions, 1):
    assert len(all_trees) == n, f"Expected {n} trees, got {len(all_trees)}"
    for i_t, tree in enumerate(all_trees):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(tree.center_x),
            "y": to_str(tree.center_y),
            "deg": to_str(tree.angle),
        })

df = pd.DataFrame(rows)
print(f"Submission shape: {df.shape}")
print(df.head(10))

In [ ]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)
df.to_csv('/home/submission/submission.csv', index=False)
print("Saved to /home/submission/submission.csv")

# Also save to experiment folder
df.to_csv('/home/code/experiments/002_simulated_annealing/submission.csv', index=False)
print("Also saved to experiment folder")

print(f"\nFinal Score: {overall_score:.6f}")